<a href="https://colab.research.google.com/github/PetchMa/Arduino_Satellite_Proj/blob/master/CSR_WAKEFIELD_data_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/weiyuanlou/JuliaCSR2D.git

Cloning into 'JuliaCSR2D'...
remote: Enumerating objects: 146, done.
remote: Counting objects: 100% (146/146), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 146 (delta 85), reused 88 (delta 38), pack-reused 0
Receiving objects: 100% (146/146), 7.89 MiB | 8.13 MiB/s, done.
Resolving deltas: 100% (85/85), done.


In [2]:
# Installation cell
%%capture
%%shell
if ! command -v julia 3>&1 > /dev/null
then
    wget -q 'https://julialang-s3.julialang.org/bin/linux/x64/1.7/julia-1.7.2-linux-x86_64.tar.gz' \
        -O /tmp/julia.tar.gz
    tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
    rm /tmp/julia.tar.gz
fi
julia -e 'using Pkg; pkg"add IJulia; precompile;"'
echo 'Done'

After you run the first cell (the the cell directly above this text), go to Colab's menu bar and select **Edit** and select **Notebook settings** from the drop down. Select *Julia 1.7* in Runtime type. You can also select your prefered harwdware acceleration (defaults to GPU). 

<br/>You should see something like this:

> ![Colab Img](https://raw.githubusercontent.com/Dsantra92/Julia-on-Colab/master/misc/julia_menu.png)

<br/>Click on SAVE
<br/>**We are ready to get going**





In [2]:
VERSION

v"1.7.2"

**The next three cells are for GPU benchmarking. If you are using this notebook for the first time and have GPU enabled, you can give it a try.** 

### Optional GPU Experiments

In [3]:
using Pkg
Pkg.add(["BenchmarkTools", "CUDA"])
using BenchmarkTools, CUDA

if has_cuda_gpu()
  print("The GPU device is:", CUDA.device())
end

The GPU device is:CuDevice(0)

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed GPUArraysCore ─────── v0.1.2
   Installed IrrationalConstants ─ v0.1.1
   Installed Adapt ─────────────── v3.4.0
   Installed TimerOutputs ──────── v0.5.21
   Installed AbstractFFTs ──────── v1.2.1
   Installed GPUCompiler ───────── v0.16.6
   Installed SpecialFunctions ──── v2.1.7
   Installed BenchmarkTools ────── v1.3.2
   Installed LLVMExtra_jll ─────── v0.0.16+0
   Installed CEnum ─────────────── v0.4.2
   Installed BFloat16s ─────────── v0.2.0
   Installed Reexport ──────────── v1.2.2
   Installed Random123 ─────────── v1.6.0
   Installed ChainRulesCore ────── v1.15.6
   Installed GPUArrays ─────────── v8.5.0
   Installed RandomNumbers ─────── v1.5.3
   Installed LogExpFunctions ───── v0.3.18
   Installed OpenSpecFun_jll ───── v0.5.5+0
   Installed Compat ────────────── v4.3.0
   Installed ExprTools ─────────── v0.1.8
   Installed InverseFunctions ──── v0.1.8
   Installed Req

In [4]:
mcpu = rand(2^10, 2^10)
@benchmark mcpu*mcpu

BenchmarkTools.Trial: 88 samples with 1 evaluation.
 Range (min … max):  53.471 ms … 72.865 ms  ┊ GC (min … max): 0.00% … 3.49%
 Time  (median):     55.827 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   56.859 ms ±  3.691 ms  ┊ GC (mean ± σ):  0.64% ± 1.67%

      █▂█  ▁▂ ▁                                                
  ▃▆▄▆████▆██▇█▄▆▁▃▆▃▃▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▃▁▁▁▁▁▁▄ ▁
  53.5 ms         Histogram: frequency by time        71.3 ms <

 Memory estimate: 8.00 MiB, allocs estimate: 2.

In [5]:
println("The CuArrray operation should be much faster (~100 times) than the CPU implemenation.")
mgpu = cu(mcpu)
@benchmark CUDA.@sync mgpu*mgpu

The CuArrray operation should be much faster (~100 times) than the CPU implemenation.


BenchmarkTools.Trial: 7897 samples with 1 evaluation.
 Range (min … max):  401.610 μs …  16.255 ms  ┊ GC (min … max): 0.00% … 46.21%
 Time  (median):     613.253 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   623.266 μs ± 263.950 μs  ┊ GC (mean ± σ):  0.29% ±  0.71%

                                  ▂▇██▆▅▃                        
  ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▄▇████████▆▅▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  402 μs           Histogram: frequency by time          761 μs <

 Memory estimate: 640 bytes, allocs estimate: 32.

In [6]:
import Pkg; Pkg.add("Plots")
import Pkg; Pkg.add("LaTeXStrings")
import Pkg; Pkg.add("Colors")
import Pkg; Pkg.add("DelimitedFiles")
import Pkg; Pkg.add("LazyGrids")
import Pkg; Pkg.add("LazyGrids")

   Resolving package versions...
   Installed JpegTurbo_jll ──────────────── v2.1.2+0
   Installed x265_jll ───────────────────── v3.5.0+0
   Installed Libmount_jll ───────────────── v2.35.0+0
   Installed libfdk_aac_jll ─────────────── v2.0.2+0
   Installed LoggingExtras ──────────────── v0.4.9
   Installed GR_jll ─────────────────────── v0.69.1+0
   Installed Opus_jll ───────────────────── v1.3.2+0
   Installed LERC_jll ───────────────────── v3.0.0+1
   Installed Xorg_xkbcomp_jll ───────────── v1.4.2+4
   Installed Measures ───────────────────── v0.3.1
   Installed RelocatableFolders ─────────── v1.0.0
   Installed Grisu ──────────────────────── v1.0.2
   Installed Formatting ─────────────────── v0.4.2
   Installed Xorg_xcb_util_wm_jll ───────── v0.4.1+1
   Installed Xorg_xcb_util_image_jll ────── v0.4.0+1
   Installed PlotUtils ──────────────────── v1.3.1
   Installed RecipesPipeline ────────────── v0.6.10
   Installed Contour ────────────────────── v0.6.2
   Installed OpenSSL ─────

In [7]:
import Pkg; Pkg.add("Distributions")
import Pkg; Pkg.add("Random")

using Distributions, Random

# Fake Gaussian beam

Np = 5000000   # Number of macro-particles
Q = 5E-9       # Total charge in C

charges = ones(Np)*Q/Np;

bb = Array{Float64, 2}(undef, 6, Np);
Random.seed!(1234)

bb[1,1:Np] = rand(Normal(0, 50E-6), Np);
bb[2,1:Np] = rand(Normal(0, 50E-6), Np);
bb[3,1:Np] = rand(Normal(0, 50E-6), Np);
bb[4,1:Np] = rand(Normal(0, 50E-6), Np);
bb[5,1:Np] = rand(Normal(0, 50E-6), Np);
bb[6,1:Np] = rand(Normal(0, 1E-6), Np);

zmin = minimum(bb[5,:])
zmax = maximum(bb[5,:])
xmin = minimum(bb[1,:])
xmax = maximum(bb[1,:])

z_b = bb[5,:]
x_b = bb[1,:];

   Resolving package versions...
   Installed DualNumbers ───────────── v0.6.8
   Installed Calculus ──────────────── v0.5.1
   Installed PDMats ────────────────── v0.11.16
   Installed Rmath_jll ─────────────── v0.3.0+0
   Installed HypergeometricFunctions ─ v0.3.11
   Installed Rmath ─────────────────── v0.7.0
   Installed StatsFuns ─────────────── v1.0.1
   Installed QuadGK ────────────────── v2.6.0
   Installed FillArrays ────────────── v0.13.5
   Installed DensityInterface ──────── v0.4.0
   Installed Distributions ─────────── v0.25.76
    Updating `~/.julia/environments/v1.7/Project.toml`
  [31c24e10] + Distributions v0.25.76
    Updating `~/.julia/environments/v1.7/Manifest.toml`
  [49dc2e85] + Calculus v0.5.1
  [b429d917] + DensityInterface v0.4.0
  [31c24e10] + Distributions v0.25.76
  [fa6b7ba4] + DualNumbers v0.6.8
  [1a297f60] + FillArrays v0.13.5
  [34004b35] + HypergeometricFunctions v0.3.11
  [90014a1f] + PDMats v0.11.16
  [1fd47b50] + QuadGK v2.6.0
  [79098fc4] + Rmath 

In [8]:
include("JuliaCSR2D/core/kick2.jl")

csr2d_kick_calc_exit (generic function with 1 method)

In [9]:
const e_charge = 1.60217663E-19 
const r_e = 2.8179402895E-15
const mec2 = 510998.94999999995

rho = 1.5
gamma = 10000.0
phi_m = 0.2

beta = (1-1/gamma^2)^(1/2)
lamb = 0.01/rho


p0c = gamma * beta * mec2

phi = 0.05
nz=201
nx=101
nxp=301
M=8

dict_ent = csr2d_kick_calc_entrance(z_b, x_b, charges,
    gamma=gamma, rho=rho, phi=phi,
    nz=nz, nx=nx, nxp=nxp, M=M, reverse_bend=false);

 Applying charge deposition...
 Computing Case A+B wake grid(s) via GPU...
366.411429 seconds (1.02 M CPU allocations: 58.357 MiB)
367.819793 seconds (508 CPU allocations: 21.891 KiB)
 Interpolating wake value at the particle positions...


In [10]:
phi_m = 0.02
lamb = 0.05
nz=201
nx=101
nxp=301
M=8

dict_exit = csr2d_kick_calc_exit(z_b, x_b, charges,
    gamma=gamma, rho=rho, phi_m=phi_m, lamb=lamb,
    nz=nz, nx=nx, nxp=nxp, M=M, reverse_bend=false)

301

In [11]:
import Pkg; Pkg.add("CSV")

   Resolving package versions...
   Installed DataValueInterfaces ───────── v1.0.0
   Installed InlineStrings ─────────────── v1.3.0
   Installed WeakRefStrings ────────────── v1.4.2
   Installed TableTraits ───────────────── v1.0.1
   Installed Tables ────────────────────── v1.10.0
   Installed SentinelArrays ────────────── v1.3.16
   Installed PooledArrays ──────────────── v1.4.2
   Installed IteratorInterfaceExtensions ─ v1.0.0
   Installed FilePathsBase ─────────────── v0.9.20
   Installed CSV ───────────────────────── v0.10.7
    Updating `~/.julia/environments/v1.7/Project.toml`
  [336ed68f] + CSV v0.10.7
    Updating `~/.julia/environments/v1.7/Manifest.toml`
  [336ed68f] + CSV v0.10.7
  [e2d170a0] + DataValueInterfaces v1.0.0
  [48062228] + FilePathsBase v0.9.20
  [842dd82b] + InlineStrings v1.3.0
  [82899510] + IteratorInterfaceExtensions v1.0.0
  [2dfb63ee] + PooledArrays v1.4.2
  [91c51154] + SentinelArrays v1.3.16
  [3783bdb8] + TableTraits v1.0.1
  [bd369af6] + Tables v1.1

In [ ]:
using CSV

CSV.write("exit.csv", dict_exit)
CSV.write("entrance.csv", dict_ent)